In [1]:
from qcutils.credential import load_provider
from qcutils.benchmarks import QASMBenchmark
from qcutils.pulse import CircAmpTimeSeries, BasisPulse
from qcutils.pulse.amptimeseries.metric import dist, min_circ_dist_in_list

from qiskit import transpile, schedule


provider = load_provider()
backend = provider.get_backend("ibm_lagos")

bm = QASMBenchmark(r"../../QASMBench", "small", num_qubits_list=list(range(8)), remove_final_measurements=True, do_transpile=False)

In [2]:
# save the current basis pulse library

bp = BasisPulse(backend)
bp.save_basis_amp_time_series_list(f"./data/basis_pulse_{backend.configuration().backend_name}.pickle")

In [3]:
def circ_list(bm, circ_name, backend, num_layouts, seed_layout = None, **transpile_args):
    import itertools, random

    if seed_layout is not None:
        random.seed(seed_layout)
    
    b_num_qubits = backend.configuration().n_qubits
    num_qubits = bm.num_qubits(circ_name)

    # randomly choose k items from all permutations
    layout_list = list(itertools.permutations(range(b_num_qubits), num_qubits))
    k = min(len(layout_list), num_layouts)
    selected_layout = []
    while len(selected_layout) < k:
        item = random.choice(layout_list)
        if item not in selected_layout:
            selected_layout.append(list(item))
    
    transpiled_circ_list = [transpile(bm.get(circ_name), backend, initial_layout=initial_layout, **transpile_args) for initial_layout in selected_layout]
    return transpiled_circ_list

In [4]:
def result(bm, backend, num_layouts, seed_layout = None, **transpile_args):
    print("Benchmark\t\tMin CD (QM)\tMin IDX Pair")

    for circ_name in bm.circ_name_list:
        try:
            transpiled_circ_list = circ_list(bm, circ_name, backend, num_layouts=num_layouts, seed_layout=seed_layout, **transpile_args)
            min_dist, min_idx_pair = min_circ_dist_in_list(transpiled_circ_list, backend, normalize=True)
            print(f"{circ_name:<24}{min_dist:.3f}\t\t{min_idx_pair}")
        except:
            continue

In [5]:
result(bm, backend, 10, seed_layout=0, seed_transpiler=0)

Benchmark		Min CD (QM)	Min IDX Pair
deutsch_n2              0.025		(4, 3)
dnn_n2                  0.039		(4, 5)
grover_n2               0.143		(4, 5)
iswap_n2                0.143		(4, 5)
quantumwalks_n2         0.125		(4, 5)
basis_change_n3         0.673		(8, 6)
fredkin_n3              0.800		(2, 5)
linearsolver_n3         0.735		(7, 1)
qaoa_n3                 0.546		(6, 8)
teleportation_n3        0.473		(1, 7)
toffoli_n3              0.096		(7, 0)
wstate_n3               0.789		(6, 8)
adder_n4                0.727		(3, 4)
basis_trotter_n4        0.895		(2, 3)
bell_n4                 0.781		(3, 4)
cat_state_n4            0.744		(8, 6)
hs4_n4                  0.545		(6, 5)
inverseqft_n4           0.000		(7, 2)
qft_n4                  0.817		(5, 3)
qrng_n4                 0.000		(7, 2)
variational_n4          0.792		(6, 5)
vqe_n4                  0.660		(2, 9)
vqe_uccsd_n4            0.858		(0, 9)
error_correctiond3_n5   0.855		(2, 6)
lpn_n5                  0.576		(4, 1)
pea_n5        